In [1]:
import numpy as np
from pandas import DataFrame, Series
import pandas as pd

# 数据聚合与分组运算
- 根据一个或者多个键，拆分pandas对象
- 计算分组摘要统计， 如计数/平均值/标准差/或用户自定义函数
- 对Dataframe的列应用各种各样的函数
- 应用组内转换或其他运算，如规格化/线性回归/排名或者选取子集等等
- 计算透视表或交叉表
- 执行分位数分析以及其他分析

# GroupBy
- pandas的groupby是lazy action，即返回的是groupby对象但是没有进行任何计算，只是含有一些有关分组键的中间数据而已。
- groupby也接受分组键列表以产生具有层次索引的结果
- group接受的参数可以是任意数组型数据，只要和df数据等长就行
- 所谓“麻烦列”会被自动剔除
- size方法返回各个子分组大小
- 分组键中的缺失值会被直接忽略
- 分组默认在axis0上进行
- 分组键会作为结果的索引，使用as_index=False取消该功能

In [8]:
# Series的groupby方法
df = DataFrame({'key':['a','b','a','a','b'], 'value':[1,2,3,4,5]})
group = df['value'].groupby(df.key)
print(group.max())

# 如果是DataFrame调用，可以通过传入列名方式简写
group = df.groupby('key')
group.max()



key
a    4
b    5
Name: value, dtype: int64


,value
key,
a,4
b,5


In [11]:
# 也可以用一个完全无关的数据进行分组
df.groupby(['c','c','c','d','d']).mean()

,value
c,2.0
d,4.5


In [12]:
# 层次索引分组
df.groupby([['c','c','c','d','d'],['e','e','f','e','f']]).mean()

value
c e    1.5
  f    3.0
d e    4.0
  f    5.0

## 对分组进行迭代
- groupby对象是一个可迭代对象
- 可以方便的转化为列表和字典

In [13]:
# name是分组键名
# grouped是各个子分组，数据类型与df相同
for name,grouped in df.groupby('key'):
    print(name)
    print(grouped)

a
  key  value
0   a      1
2   a      3
3   a      4
b
  key  value
1   b      2
4   b      5


In [16]:
# 将分组转化为字典
d = dict(list(df.groupby('key')))
d['a']


,key,value
0,a,1
2,a,3
3,a,4


## 选取一个或一组列
- 有时不需要对全部列进行分组，可以指定对哪个列进行分组
- 如果指定的是多个列组成的列表，则返回的是一个DataFrame，如果只是一个列，那么返回的是一个Series

In [9]:
df = DataFrame(np.random.randint(1,4,size=(5,4)), columns=['key1','key2','value1','value2'])
print(df)
df.groupby('key1')['value1'].mean()

   key1  key2  value1  value2
0     1     3       1       3
1     2     2       2       1
2     3     1       2       3
3     1     2       1       3
4     1     2       2       3


key1
1    1.333333
2    2.000000
3    2.000000
Name: value1, dtype: float64

In [12]:
# 或者
df['value1'].groupby(df['key1']).mean()

key1
1    1.333333
2    2.000000
3    2.000000
Name: value1, dtype: float64

In [14]:
# 对多个列分组
df.groupby('key1')[['value1','value2']].mean()

,value1,value2
key1,,
1,1.333333,3.0
2,2.000000,1.0
3,2.000000,3.0


## 通过字典或Series或者函数进行分组
- 其实就是在指定轴的索引上进行一次映射或者应用一次函数，得到分组键
- 可以混用

In [16]:
# 传入字典，指定轴，对轴应用字典映射，形成分组键
dic = {
    'key1':1, 
    'key2':1, 
    'value1':1, 
    'value2':2
}

df.groupby(dic,axis=1).count()

,1,2
0,3,1
1,3,1
2,3,1
3,3,1
4,3,1


In [21]:
# 将字典转化为Series，效果一致
df.groupby(Series(dic), axis=1).count()

,1,2
0,3,1
1,3,1
2,3,1
3,3,1
4,3,1


In [23]:
# 传入函数，根据列明长度进行分组
df.groupby(len, axis=1).count()

,4,6
0,2,2
1,2,2
2,2,2
3,2,2
4,2,2


## 指定索引级别
- 对于层次索引，指定对哪一个级别的索引进行分组


In [43]:
columns = pd.MultiIndex.from_arrays([['A','A','A','B','B'],[1,3,5,1,3]], names=['upper','lower'])
df = DataFrame(np.random.randn(4,5), columns=columns)
df.groupby(level='upper', axis=1).count()

upper,A,B
0,3,2
1,3,2
2,3,2
3,3,2


# 数据聚合
- groupby是第一步，数据聚合是第二步，就是应用在哥哥分组上并产生一个标量，例如平均值，最大值，最小值等
- df.groupby(...).agg(函数或者函数名), 函数可以是自定义函数，函数名可以是内部预定义的函数

|函数名|说明|
|:---|:---|
|count||
|sum||
|mean||
|median||
|std, var||
|min, max||
|prod|乘积|
|first, last||

- 这些内部经过优化，速度更快

In [52]:
# 对多组列应用一个聚合函数
df = DataFrame(np.random.randint(1,4,size=(5,4)), columns=['key1','key2','value1','value2'])
print(df)
df.groupby('key1')['value1','value2'].mean()

   key1  key2  value1  value2
0     1     3       2       2
1     1     3       2       3
2     1     2       3       1
3     3     3       3       3
4     2     1       1       2


,value1,value2
key1,,
1,2.333333,2.0
2,1.000000,2.0
3,3.000000,3.0


In [55]:
# 对多组列应用多个聚合函数
df.groupby('key1')['value1','value2'].agg(['mean','max','min'])

value1         value2        
          mean max min   mean max min
key1                                 
1     2.333333   3   2      2   3   1
2     1.000000   1   1      2   2   2
3     3.000000   3   3      3   3   3

In [56]:
# 为函数取名
df.groupby('key1')['value1','value2'].agg([("平均值","mean"),("最大值",'max'),("最小值",'min')])

value1         value2        
           平均值 最大值 最小值    平均值 最大值 最小值
key1                                 
1     2.333333   3   2      2   3   1
2     1.000000   1   1      2   2   2
3     3.000000   3   3      3   3   3

In [60]:
# 对不同的列应用不同的聚合函数
df.groupby('key1')['value1','value2'].agg({'value1':[("最大",'max'),("最小",'min')],'value2':[("和",'sum')]})

value2 value1   
          和     最大 最小
key1                 
1         6      3  2
2         2      1  1
3         3      3  3

## 分组级运算和转换
- 其实是分组，聚合，连接操作的快捷操作

In [76]:
# 例如想加上一列，代表各自组内平均值
# 可以分组后使用merge函数
df = DataFrame(np.random.randn(5,2),columns=['value1','value2'])
df['key1'] = ['a','a','b','b','a']
df['key2'] = ['c','d','c','d','c']
print(df)
demean = DataFrame(df.groupby('key1')['value1'].mean())
df.merge(demean, left_on='key1', right_index=True)


     value1    value2 key1 key2
0 -1.145199  0.956873    a    c
1 -1.292490  0.308645    a    d
2  0.020012 -1.078794    b    c
3 -0.034771  1.175728    b    d
4 -0.830607 -1.470460    a    c


,value1_x,value2,key1,key2,value1_y
0,-1.145199,0.956873,a,c,-1.089432
1,-1.292490,0.308645,a,d,-1.089432
4,-0.830607,-1.470460,a,c,-1.089432
2,0.020012,-1.078794,b,c,-0.007380
3,-0.034771,1.175728,b,d,-0.007380


In [80]:
# 可以直接用transform实现，其实就是将一个函数应用到各个分组，并将结果放在各自应该在的位置
# 如果返回的是一个等长数组，就对应元素赋值，如果是一个标量，就广播
df.groupby('key1')['value1'].transform(np.mean)

0   -1.089432
1   -1.089432
2   -0.007380
3   -0.007380
4   -1.089432
Name: value1, dtype: float64

## apply: 一般性的“拆分-应用-合并”
- 对每个分组应用一个自定义函数，产生任意长度的结果，然后拼接到一起

In [87]:
# 产生最大的两个值
def top(df, n=2, column='value1'):
    return df.sort_index(by=column)[-n:]

df = DataFrame(np.random.randn(5,2),columns=['value1','value2'])
df['key1'] = ['a','a','b','b','a']
df['key2'] = ['c','d','c','d','c']
print(df)
df.groupby('key1').apply(top)


     value1    value2 key1 key2
0  0.119223  2.511518    a    c
1  0.495739 -0.188446    a    d
2 -0.900091  1.426467    b    c
3 -1.485102  0.467687    b    d
4 -0.867047  1.884035    a    c


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  This is separate from the ipykernel package so we can avoid doing imports until


value1    value2 key1 key2
key1                                
a    0  0.119223  2.511518    a    c
     1  0.495739 -0.188446    a    d
b    3 -1.485102  0.467687    b    d
     2 -0.900091  1.426467    b    c

In [88]:
# 如果传入apply中的函数有额外的参数，跟在后面就可以
df.groupby('key1').apply(top, column="value2")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  This is separate from the ipykernel package so we can avoid doing imports until


value1    value2 key1 key2
key1                                
a    4 -0.867047  1.884035    a    c
     0  0.119223  2.511518    a    c
b    3 -1.485102  0.467687    b    d
     2 -0.900091  1.426467    b    c

# 分位数和桶分析
- catagory对象也可以作为groupby的输入